In [1]:
import pandas as pd
import numpy as np
import sys
from timeit import default_timer as timer

In [2]:
from surprise import Reader, Dataset, SVD, SVDpp
from surprise import accuracy
from surprise.model_selection import train_test_split

In [3]:
df = pd.read_csv('Netflix_rating_reduced.csv')

In [4]:
df.head()

,User_ID,Rating,Movie_ID
0,44937,5,3
1,439011,1,3
2,115498,3,3
3,66414,5,3
4,67315,4,3


In [5]:
data = Dataset.load_from_df(df[['User_ID','Movie_ID','Rating']],Reader())
#trainset, testset = train_test_split(data, test_size=0.3,stratify = df['User_ID'], random_state=101)
trainset = data.build_full_trainset()

In [7]:
###SVD EXPERIMENT###
algo1 = SVD(n_epochs=10, n_factors=50)
algo2 = SVD(n_epochs=30, n_factors=100)
algo3 = SVD(n_epochs=30, n_factors=150)

In [14]:
###SVD++ EXPERIMENT###
algo1pp = SVDpp(n_epochs=10, n_factors=20)
algo2pp = SVDpp(n_epochs=30, n_factors=100)
algo3pp = SVDpp(n_epochs=30, n_factors=150)

In [12]:
from surprise.model_selection import cross_validate

cross_validate(algo1, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8625  0.8622  0.8639  0.8618  0.8642  0.8629  0.0010  
Fit time          58.30   59.52   58.52   60.15   61.90   59.68   1.30    
Test time         11.70   9.71    10.62   10.73   10.59   10.67   0.63    


{'test_rmse': array([0.86248544, 0.86220026, 0.86392026, 0.86175927, 0.86421275]),
 'fit_time': (58.29920697212219,
  59.52275848388672,
  58.51758527755737,
  60.15328121185303,
  61.90442085266113),
 'test_time': (11.704397678375244,
  9.705456256866455,
  10.615008115768433,
  10.726435422897339,
  10.59330153465271)}

In [16]:
cross_validate(algo1pp, data, measures=['RMSE'], cv=3, verbose=True)

Evaluating RMSE of algorithm SVDpp on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8627  0.8629  0.8640  0.8632  0.0006  
Fit time          237.78  229.20  225.77  230.91  5.05    
Test time         193.08  175.37  170.29  179.58  9.77    


{'test_rmse': array([0.8626743 , 0.86287455, 0.86395246]),
 'fit_time': (237.77787280082703, 229.1965775489807, 225.77000331878662),
 'test_time': (193.07589268684387, 175.37117981910706, 170.28524708747864)}

In [13]:
model = SVD(n_epochs=10, verbose=True)
data = Dataset.load_from_df(df[['User_ID','Movie_ID','Rating']],Reader())
trainset, testset = train_test_split(data, test_size=0.3, random_state=10)
trainset = data.build_full_trainset()

In [17]:
model.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


In [18]:
predictions = model.test(testset)
accuracy.rmse(predictions, verbose=True)

RMSE: 0.8294


0.8294318833246187

In [19]:
df_title = pd.read_csv('Netflix_Dataset_Movie.csv')
f = ['count','mean']
df_movie_summary = df.groupby('Movie_ID')['Rating'].agg(f)
movie_benchmark = round(df_movie_summary['count'].quantile(0.7),0)
drop_movie_list = df_movie_summary[df_movie_summary['count'] < movie_benchmark].index

"\ndef Recommender(given_user_id, n_movies):\n    given_user = df_title.copy()\n    given_user = given_user.reset_index()\n    given_user = given_user[~given_user['Movie_ID'].isin(drop_movie_list)]\n    \n    given_user['Estimated_Rating'] = given_user['Movie_ID'].apply ...\n    (lambda x: model.predict(given_user_id, x).est)\n    \n    given_user = given_user.drop('Movie_ID', axis = 1)\n    given_user = given_user.sort_values('Estimated_Rating', ascending = False)\n    given_user.drop(['index'], axis = 1, inplace = True)\n    given_user.reset_index(inplace = True, drop = True)\n    return given_user.head(n_movies)\n"

In [21]:
def Recommender(given_user_id, n_movies):
    given_user = df_title.copy()
    given_user = given_user.reset_index()
    given_user = given_user[~given_user['Movie_ID'].isin(drop_movie_list)]
    
    given_user['Estimated_Rating'] = given_user['Movie_ID'].apply(lambda x: model.predict(given_user_id, x).est)
    
    given_user = given_user.drop('Movie_ID', axis = 1)
    given_user = given_user.sort_values('Estimated_Rating', ascending = False)
    given_user.drop(['index'], axis = 1, inplace = True)
    given_user.reset_index(inplace = True, drop = True)
    return given_user.head(n_movies)

In [22]:
Recommender(712664,5)

,Year,Name,Estimated_Rating
0,1954,Seven Samurai,5.000000
1,1990,The Simpsons: Treehouse of Horror,4.910361
2,1991,The Simpsons: Season 3,4.816233
3,1992,Reservoir Dogs,4.801512
4,1949,The Third Man,4.694483


In [23]:
Recommender(2643029,10)

,Year,Name,Estimated_Rating
0,1995,Pride and Prejudice,4.632754
1,1954,Seven Samurai,4.612518
2,1990,The Simpsons: Treehouse of Horror,4.580988
3,1992,Reservoir Dogs,4.561178
4,1974,The Godfather,4.524487
5,2002,The Pianist,4.519947
6,1959,North by Northwest,4.469716
7,1999,American Beauty,4.463408
8,1989,The Simpsons: Season 1,4.460580
9,1991,The Simpsons: Season 3,4.458849
